In the [previous notebook](https://nbviewer.jupyter.org/github/johnhearn/notebooks/blob/master/QuantumComputing/A%20First%20Step%20in%20Understanding%20Quantum%20Computing.ipynb) we simulated a quantum random number generator using a few lines of Julia code. We showed how quantum *superposition* can be used as a random number source. 

This time we'll simulate a quantum secure communication technique using another the mysterious quantum phenomenon known as *entanglement*. To do that we need to go a step further and work with two qubits at the same time.

As a recap these are the definitions we saw earlier. We created two qubit states representing `true` or `1` and `false` or `0`, and a NOT gate representing a rotation by 180º.

In [1]:
const ⬆ = [1., 0.] # Full upness and no downness
const ⬇ = [0., 1.] # No upness and full downness

const NOT = [0 1
             1 0];

## Qubit Pairs

In the real world particles combine to form atoms following the rules of quantum mechanics. The same physics can be used combine qubits too!

So how do we combine our qubits together? We use what is called the Kronecker product. It's not as scary as it sounds. In fact we don't really need to understand the details at all and since it's just standard linear algebra Julia has it built in!

In [2]:
⊗(x,y) = kron(x,y)

const ⬆⬆ = ⬆⊗⬆
const ⬇⬇ = ⬇⊗⬇
const ⬇⬆ = ⬇⊗⬆
const ⬆⬇ = ⬆⊗⬇;

As easily as that we have a representation of all the possible combinations of ⬇ and ⬆. 

The same thing applies to gates. By combining two NOT gates in the same way, we can flip both qubits together.

In [3]:
@assert (NOT⊗NOT)*⬆⬆ ≈ ⬇⬇
@assert (NOT⊗NOT)*⬇⬇ ≈ ⬆⬆
@assert (NOT⊗NOT)*⬇⬆ ≈ ⬆⬇
@assert (NOT⊗NOT)*⬆⬇ ≈ ⬇⬆

Following this recipe we can apply any operators we like to the individual qubits. And if, for example, we only want to change the one of the qubits then the other can be left untouched using an identity matrix, $I_2$, which is essentially a NOP gate.

In [4]:
const I₂ = [1 0
            0 1]

@assert (NOT⊗I₂)*⬆⬆ ≈ ⬇⬆

## Entangled Pairs

Now, in the world of quantum mechanics it turns out that we can also do things to a pair of qubits which cannot, even in principle, be done to the qubits individually.

I'll say that again: we can do more, much more, with two qubits combined together than we can with two separate ones. This is known as *entanglement* and is a very powerful concept, another of the keys to quantum computing. 

Take, for example, this 4x4 matrix applied to our double qubits.

In [5]:
const CNOT = [1 0 0 0
              0 1 0 0
              0 0 0 1
              0 0 1 0]

@assert CNOT*(⬇⊗⬇) != ⬇⊗⬇
@assert CNOT*(⬆⊗⬇) != ⬇⊗⬆

If the first qubit is ⬇ then the gate does nothing, however if the first qubit is ⬆ then it flips the second qubit. Its a NOT gate but in some way dependent (or controlled) on the first qubit. It's called a [CNOT gate](https://en.wikipedia.org/wiki/Controlled_NOT_gate).

This gate *cannot* be constructed as the combination of multiple single qubit gates however it *can* be constructed using a quantum computers in the real world. Quantum physics allows qubit pairs to seemingly do the impossible! 

# Quantum communication

Quantum computing is often hailed as the end of internet security because it could, theoretically, deconstruct the keys used for HTTPS communication. 

On the other hand quantum effects can also be the ultimate solution for encrypted communication using pairs of entangled quantum particles. In fact this is [already](https://www.newscientist.com/article/2229673-china-has-developed-the-worlds-first-mobile-quantum-satellite-station/) happening!

This is how it works. We take a pair of qubits (which in reality would, for example, be particles of light) and combine them with our CNOT gate. Send one of the qubits (particles) to a friend. 

They do something to it to encode a value and then send it back. 

The fact that the two qubits were entangled means that the qubit our friend sends back can only be decoded reliably by us because we have the other qubit. What's more it can only be decoded once because the measurement process destroys the entanglement. Any kind of eavesdropping is immediately detectable.

How does all of this work in code? First we take a pair of qubits, lets say ⬆⬆, apply a $\sqrt{NOT}$ to the first and then apply a CNOT to both, like this.

In [6]:
Q = CNOT * (√NOT⊗I₂) * ⬆⬆;

The variable $Q$ now represents an entangled qubit pair! Notice that the gates are applied in *reverse* order, that it, they are evaluated right to left.

The next step is to send the qubit to a friend who does something to it and then sends it back. (In the real quantum world, qubits could be photons transmitted and passed through lenses independently, even though they remain entangled in principle.)

Let's say we got our qubit back. We decode it by just reversing the operations we did earlier.

In [7]:
Q = (√NOT⊗I₂) * CNOT * Q;

We now have our qubit pair back with whatever value our friend wanted to send us tucked inside it. To get it we just perform the measurement. Here again we apply a measurement function which, as in the first part, we can treat as a black box. 

In [8]:
function measure2(r)
    M₀ = ⬆*⬆'
    [abs(r'*M*r) < rand() for M in [M₀⊗I₂, I₂⊗M₀]]
end

@assert measure2(Q) == [1, 0]

We have received two bits of information `10`. In fact two bits for the price of one, I guess that's why its called [*superdense*](https://en.wikipedia.org/wiki/Superdense_coding)? Anyway, what if our friend wanted to send some other information, like `01`? They just apply the NOT gate to their qubit. Let's try it.

In [9]:
Q = CNOT*(√NOT⊗I₂)*⬆⬆ # Qubit pair, send first qubit to friend

Q = (NOT⊗I₂)*Q # Our friend applies a NOT gate to their qubit

Q = (√NOT⊗I₂)*CNOT*Q # We receive the single qubit back and decode the pair

@assert measure2(Q) == [0, 1]

What about `00` and `11`? For that we need to introduce another gate, [Z](https://en.wikipedia.org/wiki/Quantum_logic_gate#Pauli-Z_('%22%60UNIQ--postMath-00000028-QINU%60%22')_gate), which is just another type of rotation. With this type of gate our friend can now encode all four possibilities by choosing the appropriate gate and applying it to their single qubit:

In [10]:
const Z = [1  0
           0 -1]

Φs = [I₂, NOT, Z, NOT*Z] # The four encodings that our friend can apply

[measure2((√NOT⊗I₂) * CNOT * (Φ⊗I₂) * CNOT * (√NOT⊗I₂) * ⬆⬆) for Φ in Φs]

4-element Array{Array{Bool,1},1}:
 [1, 0]
 [0, 1]
 [0, 0]
 [1, 1]

## Wrapping up

We've seen how pairs of entangled particles can be created and used to send information and we simulated that in code.

For a deeper understanding take a look at [Michael Nielsen](https://www.youtube.com/channel/UCoSlN8Gh4W8sfgB-Sf_cG4Q)'s YouTube videos "[Superdense coding: how to send two bits using one qubit](https://www.youtube.com/watch?v=w5rCn593Dig)" and "[What's so special about entangled states anyway?](https://www.youtube.com/watch?v=aRglXdLI7KY&list=PL1826E60FD05B44E4&index=14)", which goes into more detail and with more technical notations.

What happens when we combine more than two qubits? Well, it opens up a whole world of new algorithms and that's what we'll start exploring in the next parts...